# Video Game Ratings Predictor
> Testing with an LLM

In [8]:
pip install openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
from openai import OpenAI
import matplotlib.pyplot as plt
import math

In [29]:
df = pd.read_csv("imdb-videogames.csv")
df = df.dropna()
df = df.reset_index()
df.head(20)

,index,Unnamed: 0,name,url,year,certificate,rating,votes,plot,Action,Adventure,Comedy,Crime,Family,Fantasy,Mystery,Sci-Fi,Thriller
0,0,0,Spider-Man,https://www.imdb.com/title/tt5807780/?ref_=adv...,2018.0,T,9.2,"20,759","When a new villain threatens New York City, Pe...",True,True,False,False,False,True,False,False,False
1,1,1,Red Dead Redemption II,https://www.imdb.com/title/tt6161168/?ref_=adv...,2018.0,M,9.7,"35,703",Amidst the decline of the Wild West at the tur...,True,True,False,True,False,False,False,False,False
2,2,2,Grand Theft Auto V,https://www.imdb.com/title/tt2103188/?ref_=adv...,2013.0,M,9.5,"59,986",Three very different criminals team up for a s...,True,False,False,True,False,False,False,False,False
3,3,3,God of War,https://www.imdb.com/title/tt5838588/?ref_=adv...,2018.0,M,9.6,"26,118","After wiping out the gods of Mount Olympus, Kr...",True,True,False,False,False,False,False,False,False
4,4,4,Uncharted 4: A Thief's End,https://www.imdb.com/title/tt3334704/?ref_=adv...,2016.0,T,9.5,"28,722",Thrown back into the dangerous underworld he'd...,True,True,False,False,False,False,False,False,False
5,5,5,The Last of Us: Part II,https://www.imdb.com/title/tt6298000/?ref_=adv...,2020.0,M,8.5,"30,460","Five years after the events of The Last of Us,...",True,True,False,False,False,False,False,False,False
6,6,6,Horizon Forbidden West,https://www.imdb.com/title/tt12496904/?ref_=ad...,2022.0,T,9.2,"2,979",Aloy treks into an arcane region and faces new...,True,True,False,False,False,False,True,False,False
7,7,7,The Last of Us,https://www.imdb.com/title/tt2140553/?ref_=adv...,2013.0,M,9.7,"60,590","In a hostile, post-pandemic world, Joel and El...",True,True,False,False,False,False,False,False,False
8,8,8,Detroit: Become Human,https://www.imdb.com/title/tt5158314/?ref_=adv...,2018.0,M,9.2,"16,907",Take control of three androids in their quest ...,True,True,False,False,False,False,False,False,False
9,9,9,Death Stranding,https://www.imdb.com/title/tt5807606/?ref_=adv...,2019.0,M,8.8,"8,136",Deliveryman Sam Porter must travel across a ra...,True,True,False,False,False,False,False,False,False


In [30]:
video_games = df.drop(["url"], axis = 1)
no_ratings = df.drop(["url", "rating"], axis = 1)
video_games.head()

,index,Unnamed: 0,name,year,certificate,rating,votes,plot,Action,Adventure,Comedy,Crime,Family,Fantasy,Mystery,Sci-Fi,Thriller
0,0,0,Spider-Man,2018.0,T,9.2,"20,759","When a new villain threatens New York City, Pe...",True,True,False,False,False,True,False,False,False
1,1,1,Red Dead Redemption II,2018.0,M,9.7,"35,703",Amidst the decline of the Wild West at the tur...,True,True,False,True,False,False,False,False,False
2,2,2,Grand Theft Auto V,2013.0,M,9.5,"59,986",Three very different criminals team up for a s...,True,False,False,True,False,False,False,False,False
3,3,3,God of War,2018.0,M,9.6,"26,118","After wiping out the gods of Mount Olympus, Kr...",True,True,False,False,False,False,False,False,False
4,4,4,Uncharted 4: A Thief's End,2016.0,T,9.5,"28,722",Thrown back into the dangerous underworld he'd...,True,True,False,False,False,False,False,False,False


In [45]:
client = OpenAI(
   base_url = "http://dh-dgxh100-2.hpc.msoe.edu:8000/v1",
   api_key = "not_used" # this field needs to be included but is ignored
)

def make_llm_call(review, client):
    message = "Predict the rating for this video game out of 100:\n" + str(review)
    response = client.chat.completions.create(
        model="meta/llama-3.3-70b-instruct",
        messages=[
                {"role": "system", "content": "Only reply with an integer between 0 and 100."}, #commented out, but you might want to include it
                {"role": "user", "content": message}
            ],
        max_tokens=2, #can make larger, but you only are likely only looking for "Fresh" or "Rotten" as response
        stream=False
    )
    return response.choices[0].message.content

In [51]:
preds = []
actual = []
for i in range(500):
    rating = no_ratings.loc[i]
    # print(rating)
    # print(message)
    response = make_llm_call(rating, client)
    # print(response)
    preds.append(int(response) / 10)
    actual.append(float(video_games.loc[i, "rating"]))
    if ( i % 20 == 0):
        print(i)
    # print(str(preds[i]) + " " + str(actual[i]))
print("done")

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
done


In [53]:
total_difference = 0.0
for i in range(len(preds)):
    total_difference = round(total_difference + abs(round(float(preds[i]) - float(actual[i]), 1)), 1)
    # print(total_difference)
    
print(str(round(total_difference / 500, 2)))

0.7
